In [0]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

In [47]:
# transform=transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
#                               transforms.RandomVerticalFlip(p=0.5),
#                               transforms.ToTensor(),
#                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
transform=transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset=torchvision.datasets.CIFAR10(root='./data', train=True,
                                    download=True, transform=transform)
trainloader=torch.utils.data.DataLoader(trainset, batch_size=5,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=0)

classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

Files already downloaded and verified
Files already downloaded and verified


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, (3,3))
        self.batch1=nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, (3,3))
        self.batch2=nn.BatchNorm2d(32)
        self.drop1=nn.Dropout2d(p=0.2)
        self.conv3=nn.Conv2d(32,64,(3,3))
        self.batch3=nn.BatchNorm2d(64)
        self.conv4=nn.Conv2d(64,64,(3,3))
        self.batch4=nn.BatchNorm2d(64)
        self.drop2=nn.Dropout2d(p=0.3)
        self.conv5=nn.Conv2d(64,128,(3,3))
        self.batch5=nn.BatchNorm2d(128)
        self.conv6=nn.Conv2d(128,128,(3,3))
        self.batch6=nn.BatchNorm2d(128)
        self.drop3=nn.Dropout2d(p=0.4)
        self.fc1=nn.Linear(128*(4*4), 10)
        self.pool=nn.MaxPool2d((2,2), 2)
        self.pad=nn.ZeroPad2d(1)

    def forward(self, x):
        x = self.batch1(self.pad(F.elu(self.conv1(x))))
        x = self.batch2(self.pad(F.elu(self.conv2(x))))
        x = self.drop1(self.pool(x))
        x = self.batch3(self.pad(F.elu(self.conv3(x))))
        x = self.batch4(self.pad(F.elu(self.conv4(x))))
        x = self.drop2(self.pool(x))
        x = self.batch5(self.pad(F.elu(self.conv5(x))))
        x = self.batch6(self.pad(F.elu(self.conv6(x))))
        x = self.drop3(self.pool(x))
        x = x.view(-1, 128*(4*4))
        x = self.fc1(x)
        return x

net = Net()

In [49]:
# Classification Cross-Entropy loss and SGD with momentum
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adagrad(net.parameters(),weight_decay=10**(-4))

cuda:0


Training my network here.

In [50]:
for epoch in range(100): # no of epochs
    running_loss=0.0
    for i,data in enumerate(trainloader,0):
          inputs, labels = data[0].to(device), data[1].to(device)
          optimizer.zero_grad()
          outputs=net(inputs)
          loss=criterion(outputs,labels)
          loss.backward()
          optimizer.step()
          running_loss+=loss.item()
          if i%2000==1999:
                if i%12000==11999: running_loss=0
                #   print('Epoch:',epoch+1,', loss:',running_loss/2000)
                
    correct=0
    total=0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs=net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print("Epoch=",epoch+1,", Accuracy=",correct*100/total)
print('Model trained :)')

Epoch= 1 , Accuracy= 42.54
Epoch= 2 , Accuracy= 51.11
Epoch= 3 , Accuracy= 55.28
Epoch= 4 , Accuracy= 60.24
Epoch= 5 , Accuracy= 60.87
Epoch= 6 , Accuracy= 63.13
Epoch= 7 , Accuracy= 64.23
Epoch= 8 , Accuracy= 65.62
Epoch= 9 , Accuracy= 64.69
Epoch= 10 , Accuracy= 67.68
Epoch= 11 , Accuracy= 67.83
Epoch= 12 , Accuracy= 68.0
Epoch= 13 , Accuracy= 68.38
Epoch= 14 , Accuracy= 69.11
Epoch= 15 , Accuracy= 69.02
Epoch= 16 , Accuracy= 69.68
Epoch= 17 , Accuracy= 70.12
Epoch= 18 , Accuracy= 69.76
Epoch= 19 , Accuracy= 70.21
Epoch= 20 , Accuracy= 70.5
Epoch= 21 , Accuracy= 70.29
Epoch= 22 , Accuracy= 70.48
Epoch= 23 , Accuracy= 70.88
Epoch= 24 , Accuracy= 71.21
Epoch= 25 , Accuracy= 70.4
Epoch= 26 , Accuracy= 71.34
Epoch= 27 , Accuracy= 70.93
Epoch= 28 , Accuracy= 71.68
Epoch= 29 , Accuracy= 71.73
Epoch= 30 , Accuracy= 72.04
Epoch= 31 , Accuracy= 71.51
Epoch= 32 , Accuracy= 71.7
Epoch= 33 , Accuracy= 71.58
Epoch= 34 , Accuracy= 71.62
Epoch= 35 , Accuracy= 72.2
Epoch= 36 , Accuracy= 72.14
Epoch=

KeyboardInterrupt: ignored

In [0]:
# torch.save(net.state_dict(), './cifar2.pth')